In [22]:
#Final result:
act_cust_df_grg

,Date,Active Customers,Total balance of active people,Number of people logged into
0,2022-12-08,103,12813904613,61.0
1,2022-12-09,133,5537777339,57.0
2,2022-12-10,158,32468546947,76.0
3,2022-12-11,177,7994892601,71.0
4,2022-12-12,198,7595695963,75.0
5,2022-12-13,209,7119738966,63.0
6,2022-12-14,227,5126693560,74.0
7,2022-12-15,239,13948961823,72.0
8,2022-12-16,254,6182853544,53.0
9,2022-12-17,264,6317322108,71.0


# comment:
### اول دیتافرم رو میسازیم و تمام تاریخ های موجود رو لیست میکنیم. سپس برای هر تاریخ، برای هر مشتری، چک میکنیم که از آن تاریخ تا سی روز قبل تراکنش و یا لاگینی داشته یا نه
### اگر داشته در یک تاپل، تارخ، آیدی مشتری و مقدار موجود در حسابش در آن روز را ذخیره میکنیم و سپس آنها را در لیست های جداگانه ریخته و دیتافرم آن ها را با گروه بندی بر اساس تاریخ میسازیم.

### برای تعداد لاگین های تمام مشتریان در هر تاریخ نیز دیتافرم لاگین را بر اساس تاریخ ها گروه بندی میکنیم و برای تاریخ 19 ام نیز 
### چون دیتایی در لاگین نداریم مقدار خالی قرار میدهیم

In [1]:
import pandas as pd
import datetime

In [2]:
trxdf= pd.read_csv('trx.csv')
trxdf['transaction_date'] = pd.to_datetime(trxdf['transaction_date']).dt.date

In [3]:
logindf= pd.read_csv('login.csv')
logindf['created_date'] = pd.to_datetime(logindf['created_date']).dt.date

In [4]:
#make list off all dates that we have in our datas.
all_dates_list= trxdf['transaction_date'].value_counts().keys().tolist()
all_dates_list.sort()

In [6]:
active_list=[] #to catch info with their balance
check_list=[] #to avoid adding duplicate info
for end_date in all_dates_list:
    start_date = end_date - datetime.timedelta(days=30)
    for customer in trxdf['customer_id']:
        date = trxdf.transaction_date[trxdf['customer_id'] == customer].tolist()
        balance = trxdf.running_balance_drived[(trxdf['customer_id'] == customer)
                                               & (trxdf['transaction_date'] == end_date) ].sum()
        for date in date:
            if start_date <= date <= end_date:
                if(end_date,customer) not in check_list:
                    check_list.append((end_date,customer))
                    active_list.append((end_date,customer,balance))

In [7]:
for end_date in all_dates_list:
    start_date = end_date - datetime.timedelta(days=30)
    for customer in logindf['customer_id']:
        date = logindf.created_date[logindf['customer_id'] == customer].tolist()
        for date in date:
            if start_date <= date <= end_date:
                if (end_date,customer) not in check_list:
                    check_list.append((end_date,customer))
                    active_list.append((end_date,customer,balance))

In [8]:
dates=[]
costumers = []
balances = []
for i in active_list:
    date= i[0]
    costumer = i[1]
    balance = i[2]
    dates.append(date)
    costumers.append(costumer)
    balances.append(balance)

In [9]:
act_cust_df = pd.DataFrame(list(zip(dates, costumers)),
               columns =['Date', 'Active Customers'])

In [10]:
act_cust_df_grg = act_cust_df.groupby(['Date'], as_index = False).count()

In [11]:
act_cust_df_grg

,Date,Active Customers
0,2022-12-08,103
1,2022-12-09,133
2,2022-12-10,158
3,2022-12-11,177
4,2022-12-12,198
5,2022-12-13,209
6,2022-12-14,227
7,2022-12-15,239
8,2022-12-16,254
9,2022-12-17,264


In [12]:
act_cust_df_blnc = pd.DataFrame(list(zip(dates, balances)),
               columns =['Date', 'Balances'])

In [13]:
act_cust_df_blnc_grp = act_cust_df_blnc.groupby(['Date'], as_index = False).sum()

In [14]:
act_cust_df_blnc_grp

,Date,Balances
0,2022-12-08,12813904613
1,2022-12-09,5537777339
2,2022-12-10,32468546947
3,2022-12-11,7994892601
4,2022-12-12,7595695963
5,2022-12-13,7119738966
6,2022-12-14,5126693560
7,2022-12-15,13948961823
8,2022-12-16,6182853544
9,2022-12-17,6317322108


In [15]:
#check and delete if it is not a dupicated rows for each customer in one day
login_singles = logindf.drop_duplicates(
  subset = ['customer_id', 'created_date'],
  keep = 'last').reset_index(drop = True)

In [16]:
login_singles_grp = login_singles.groupby(['created_date'], as_index = False).count()

In [17]:
login_singles_grp

,created_date,customer_id,count_login
0,2022-12-08,61,61
1,2022-12-09,57,57
2,2022-12-10,76,76
3,2022-12-11,71,71
4,2022-12-12,75,75
5,2022-12-13,63,63
6,2022-12-14,74,74
7,2022-12-15,72,72
8,2022-12-16,53,53
9,2022-12-17,71,71


In [18]:
#Append 'nan' for last date of list because there isn't any login log info for that time.
nummberOfLogin = login_singles_grp['customer_id'].values.tolist()
nummberOfLogin.append(None)

In [19]:
act_cust_df_grg['Total balance of active people'] = act_cust_df_blnc_grp['Balances'].values
act_cust_df_grg['Number of people logged into'] = nummberOfLogin

In [20]:
act_cust_df_grg

,Date,Active Customers,Total balance of active people,Number of people logged into
0,2022-12-08,103,12813904613,61.0
1,2022-12-09,133,5537777339,57.0
2,2022-12-10,158,32468546947,76.0
3,2022-12-11,177,7994892601,71.0
4,2022-12-12,198,7595695963,75.0
5,2022-12-13,209,7119738966,63.0
6,2022-12-14,227,5126693560,74.0
7,2022-12-15,239,13948961823,72.0
8,2022-12-16,254,6182853544,53.0
9,2022-12-17,264,6317322108,71.0
